<a href="https://colab.research.google.com/github/AlexMontgomerie/deepLearning/blob/master/denoise_additive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content
#!rm -rf deepLearning
!git clone https://github.com/AlexMontgomerie/deepLearning
%cd deepLearning
!git pull origin master
!pip install gputil
!pip install psutil
!pip install humanize
# Taken from
# https://stackoverflow.com/questions/48750199/google-colaboratory-misleading-information-about-its-gpu-only-5-ram-available
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# Colab only provides one GPU and it is not always guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

/content
fatal: destination path 'deepLearning' already exists and is not an empty directory.
/content/deepLearning
From https://github.com/AlexMontgomerie/deepLearning
 * branch            master     -> FETCH_HEAD
Already up to date.
('RAM Free: 12.9 GB', ' | Proc size: 150.1 MB')
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [2]:
from common import *
!./setup.sh

Using TensorFlow backend.


In [3]:
from denoise_network import *
from get_data import get_data

EPOCHS = 50

seqs_train, seqs_test = get_data()

# get traning data

denoise_generator     = DenoiseHPatches(seqs_train, batch_size=500)
denoise_generator_val = DenoiseHPatches(seqs_test, batch_size=500)

100%|██████████| 40/40 [00:43<00:00,  1.31s/it]


In [0]:

import sys
import json
import os
import glob
import keras
from keras import regularizers
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda, Reshape
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Input, UpSampling2D, concatenate, Subtract
import time
import tensorflow as tf
import numpy as np
import cv2
import random
from read_data import HPatches, DataGeneratorDesc, hpatches_sequence_folder, DenoiseHPatches, tps
from utils import generate_desc_csv, plot_denoise, plot_triplet
import matplotlib.pyplot as plt

def get_denoise_model_additive(shape):

  inputs  = Input(shape)
  depth1  = 32
  conv1_1 = Conv2D(depth1, 1, padding = 'same', kernel_initializer = 'he_normal')(inputs)
  
  # convolution layers
  conv1_2  = Conv2D(depth1, 2 , activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1_1)
  conv1_3  = Conv2D(depth1, 3 , activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1_1)
  conv1_5  = Conv2D(depth1, 5 , activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1_1)
  conv1_7  = Conv2D(depth1, 7 , activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1_1)
  conv1_9  = Conv2D(depth1, 9 , activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1_1)
  conv1_11 = Conv2D(depth1, 11, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1_1)
  
  # network
  net1 = Subtract()([conv1_1, conv1_2])
  net1 = BatchNormalization()(net1)
  net1 = Subtract()([conv1_1, conv1_3])
  net1 = BatchNormalization()(net1)
  net1 = Subtract()([conv1_1, conv1_5])
  net1 = BatchNormalization()(net1)
  net1 = Subtract()([conv1_1, conv1_7])
  net1 = BatchNormalization()(net1)
  net1 = Subtract()([conv1_1, conv1_9])
  net1 = BatchNormalization()(net1)
  net1 = Subtract()([conv1_1, conv1_11])
  net1 = BatchNormalization()(net1)  
  
  # convolution layers
  depth2 = 16
  conv2_1  = Conv2D(depth2, 1, padding = 'same', kernel_initializer = 'he_normal')(net1)
  conv2_2  = Conv2D(depth2, 2 , activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2_1)
  conv2_3  = Conv2D(depth2, 3 , activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2_1)
  conv2_5  = Conv2D(depth2, 5 , activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2_1)
  conv2_7  = Conv2D(depth2, 7 , activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2_1)
  conv2_9  = Conv2D(depth2, 9 , activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2_1)
  conv2_11 = Conv2D(depth2, 11, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2_1)
  
  # network
  net2 = Subtract()([conv2_1, conv2_2])
  net2 = BatchNormalization()(net2)
  net2 = Subtract()([conv2_1, conv2_3])
  net2 = BatchNormalization()(net2)
  net2 = Subtract()([conv2_1, conv2_5])
  net2 = BatchNormalization()(net2)
  net2 = Subtract()([conv2_1, conv2_7])
  net2 = BatchNormalization()(net2)
  net2 = Subtract()([conv2_1, conv2_9])
  net2 = BatchNormalization()(net2)
  net2 = Subtract()([conv2_1, conv2_11])
  net2 = BatchNormalization()(net2)  
  
  # convolution layers
  depth3 = 8
  conv3_1  = Conv2D(depth3, 1, padding = 'same', kernel_initializer = 'he_normal')(net1)
  conv3_2  = Conv2D(depth3, 2 , activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3_1)
  conv3_3  = Conv2D(depth3, 3 , activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3_1)
  conv3_5  = Conv2D(depth3, 5 , activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3_1)
  conv3_7  = Conv2D(depth3, 7 , activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3_1)
  conv3_9  = Conv2D(depth3, 9 , activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3_1)
  conv3_11 = Conv2D(depth3, 11, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3_1)
  
  # network
  net3 = Subtract()([conv3_1, conv2_2])
  net3 = BatchNormalization()(net3)
  net3 = Subtract()([conv3_1, conv3_3])
  net3 = BatchNormalization()(net3)
  net3 = Subtract()([conv3_1, conv3_5])
  net3 = BatchNormalization()(net3)
  net3 = Subtract()([conv3_1, conv3_7])
  net3 = BatchNormalization()(net3)
  net3 = Subtract()([conv3_1, conv3_9])
  net3 = BatchNormalization()(net3)
  net3 = Subtract()([conv3_1, conv3_11])
  net3 = BatchNormalization()(net3)  
  
  net = Conv2D(1, 1, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(net3)
  
  return Model(inputs = inputs, outputs = net)

In [0]:
# optimiser
opt  = opt = keras.optimizers.nadam()

# get model
shape = (32, 32, 1)
denoise_model = get_denoise_model_additive(shape)

# callbacks
callbacks = [
    keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, verbose = 3, mode= 'auto'),
    keras.callbacks.ModelCheckpoint('data/denoise_model.weights.{epoch:02d}-{val_loss:.2f}.hdf5', verbose=1, save_best_only=True)
]

denoise_model.compile(loss='mean_absolute_error', optimizer=opt, metrics=['accuracy'])
denoise_history = denoise_model.fit_generator(generator=denoise_generator, epochs=EPOCHS, callbacks=callbacks,
                                              verbose=1, validation_data=denoise_generator_val)

Epoch 1/50
3117/3117 [==============================] - 540s 173ms/step - loss: 22.8066 - acc: 0.0411 - val_loss: 6.1367 - val_acc: 0.0613

Epoch 00001: val_loss improved from inf to 6.13668, saving model to data/denoise_model.weights.01-6.14.hdf5
Epoch 2/50
3117/3117 [==============================] - 536s 172ms/step - loss: 6.3540 - acc: 0.0590 - val_loss: 5.8579 - val_acc: 0.0654

Epoch 00002: val_loss improved from 6.13668 to 5.85790, saving model to data/denoise_model.weights.02-5.86.hdf5
Epoch 3/50
3117/3117 [==============================] - 534s 171ms/step - loss: 6.1618 - acc: 0.0610 - val_loss: 5.7704 - val_acc: 0.0675

Epoch 00003: val_loss improved from 5.85790 to 5.77045, saving model to data/denoise_model.weights.03-5.77.hdf5
Epoch 4/50
3117/3117 [==============================] - 534s 171ms/step - loss: 5.9290 - acc: 0.0647 - val_loss: 5.5044 - val_acc: 0.0735

Epoch 00004: val_loss improved from 5.77045 to 5.50442, saving model to data/denoise_model.weights.04-5.50.hdf5